# MA4402 Proyecto final 2025
## Internal DLA con branching random walk
## Integrantes

- Francisco Loyola
- Pascal Steiner

# IMPORTACIONES

In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import Image, display

# CLASE:

In [2]:
movimientos = ((1,0), (-1,0), (0,1), (0,-1))

def exp(tasa):
    return np.random.exponential(1.0 / tasa)

class BRW:
    def __init__(self, tasa_salto, tasa_rep, prob_vivir, posiciones):
        self.tasa_rep = float(tasa_rep)
        self.tasa_salto = float(tasa_salto)
        self.t = 0.0
        self.prob_vivir = float(prob_vivir)
        if posiciones is None:
            posiciones = [(0, 0)]
        self.posiciones = list(posiciones)

    def reproducir_morir(self):
        particulas = len(self.posiciones)
        if particulas == 0:
            return self.posiciones
        else:
            particula = random.randint(0, particulas - 1)
            u = random.uniform(0,1)
            if u < self.prob_vivir:
                self.posiciones.append(self.posiciones[particula])
            else:
                self.posiciones.pop(particula)
            return self.posiciones

    def saltar(self):
        particulas = len(self.posiciones)
        if particulas == 0:
            return self.posiciones
        else:
            particula = random.randint(0, particulas - 1)
            movimiento = random.randint(0, 3)
            i, j = movimientos[movimiento]
            self.posiciones[particula] = (self.posiciones[particula][0] + i, self.posiciones[particula][1] + j)
            return self.posiciones

    def avanza(self):
        particulas = len(self.posiciones)
        if particulas == 0:
            return self.posiciones
        u = random.uniform(0,1)
        if u < self.tasa_salto/(self.tasa_salto + self.tasa_rep):
            return self.saltar()
        else:
            return self.reproducir_morir()
     
    def avanzar(self):
        particulas = len(self.posiciones)
        if particulas == 0:
            return list(self.posiciones)
        self.t += exp(particulas * (self.tasa_salto + self.tasa_rep))
        return list(self.avanza())

    def simular(self, T_max):
      simulacion = [(list(self.posiciones),self.t)]
      while self.t < T_max:
        simulacion.append((self.avanzar(),self.t))
      return simulacion


# FUNCION ANIMADORA:

In [3]:
def animar_trayectoria(trayectoria, interval_ms=200):
    fig, ax = plt.subplots()
    scat = ax.scatter([], [])
    ax.set_aspect('equal', adjustable='box')
    ax.grid(True, linestyle='--', alpha=0.3)

    xs_all = [p[0] for pos, _ in trayectoria for p in pos]
    ys_all = [p[1] for pos, _ in trayectoria for p in pos]
    if xs_all and ys_all:
        pad = 1
        ax.set_xlim(min(xs_all)-pad, max(xs_all)+pad)
        ax.set_ylim(min(ys_all)-pad, max(ys_all)+pad)
    else:
        ax.set_xlim(-2, 2)
        ax.set_ylim(-2, 2)

    if trayectoria:
        pos0, t0 = trayectoria[0]
        xs0 = [p[0] for p in pos0]
        ys0 = [p[1] for p in pos0]
        scat.set_offsets(list(zip(xs0, ys0)))
        scat.set_sizes([36]*len(xs0))
        ax.set_title(f"t = {t0:.3f}  |  n = {len(pos0)}")

    def update(frame):
        pos, t = trayectoria[frame]
        xs = [p[0] for p in pos]
        ys = [p[1] for p in pos]
        scat.set_offsets(list(zip(xs, ys)))
        scat.set_sizes([36]*len(xs))
        ax.set_title(f"t = {t:.3f}  |  n = {len(pos)}")
        return scat,

    ani = FuncAnimation(
        fig, update, frames=len(trayectoria),
        interval=interval_ms, blit=False, repeat=False
    )
    return ani


# SIMULACION:

In [ ]:
brw = BRW(1.0, 0.2, 0.8, None)
tiempo_simulacion = 50
trayectoria = brw.simular(tiempo_simulacion)
ani = animar_trayectoria(trayectoria, interval_ms=150)
ani.save("brw.gif", writer=PillowWriter(fps=10))
plt.close(ani._fig)
display(Image(filename="brw.gif"))
